In [10]:
import pandas as pd
import json
import re

In [50]:
keywords = {
        "beautician" : ["beautician", "beauty"],
        "electrician" : ["electrician", "electricity"],
        "carpenter" : ["carpenter"],
        "nurse" : ["nurse", "hospital"],
        "pharmacist" : ["pharmacist", "medicine"],
        "receptionist" : ["receptionist", "reception"],
        "security_guard" : ["secuirty", "guard"],
        "software_engineer" : ["software", "engineer"],
    }

In [51]:
# Count the number of gendered words in each job advertisement
dataDirectory = './data/job_advertisements/en/'
wikiJobData = json.load(open('./data/job_advertisements/en/wikijob/wikijob.json', 'r'))
cleanedDataDictionary = './data/job_advertisements/en/cleaned/'

occupations = list(keywords)
for occupation in occupations:
    df = pd.DataFrame(index=[], columns=['all_sentences'])
    jsonOpen = open(dataDirectory + occupation + '.json', 'r')
    jobAdvertisements = json.load(jsonOpen)
    getIndex = 1
    dfIndex = 1
    for jobAdvertisement in jobAdvertisements:
        if jobAdvertisement.get(str(getIndex)):
            if jobAdvertisement[str(getIndex)]['title'] is None or jobAdvertisement[str(getIndex)]['description'] is None:
                getIndex += 1
                continue
            cleaned_title = re.sub(' +', ' ', jobAdvertisement[str(getIndex)]['title'].replace('\n','').replace('(',' ').replace(',',' ').replace('-',' ').replace(')',' ').strip().lower())
            cleaned_description = re.sub(' +', ' ', jobAdvertisement[str(getIndex)]['description'].replace('\n','').replace('(',' ').replace(',',' ').replace('-',' ').replace(')',' ').strip().lower())
            all_sentences =  cleaned_title + ' ' + cleaned_description

            validateFlug = False
            for keyword in keywords.get(occupation):
                if keyword in all_sentences:
                    validateFlug = True
            if validateFlug:
                record = pd.DataFrame({
                    'all_sentences': all_sentences
                }, index=[str(dfIndex)])
                df = pd.concat([df, record])
                dfIndex += 1
        getIndex += 1
    
    getIndex = 1
    for wikiJobAdvertisement in wikiJobData:
        if wikiJobAdvertisement.get(str(getIndex)) and wikiJobAdvertisement[str(getIndex)]['occupation'] == occupation:
            if wikiJobAdvertisement[str(getIndex)]['title'] is None or wikiJobAdvertisement[str(getIndex)]['description'] is None :
                continue
            cleaned_title = re.sub(' +', ' ', wikiJobAdvertisement[str(getIndex)]['title'].replace('\n','').replace('(',' ').replace(',',' ').replace('-',' ').replace(')',' ').strip().lower())
            cleaned_description = re.sub(' +', ' ', wikiJobAdvertisement[str(getIndex)]['description'].replace('\n','').replace('(',' ').replace(',',' ').replace('-',' ').replace(')',' ').strip().lower())
            all_sentences =  cleaned_title + ' ' + cleaned_description
            
            validateFlug = False
            for keyword in keywords.get(occupation):
                if keyword in cleaned_title:
                    validateFlug = True
                    break
            if validateFlug:
                record = pd.DataFrame({
                    'all_sentences': all_sentences
                }, index=[str(dfIndex)])
                df = pd.concat([df, record])
                dfIndex += 1
        getIndex += 1
    
    df.to_json(cleanedDataDictionary + occupation + '.json', force_ascii=False, orient = 'records')
